In [1]:
import pyspark

event_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferSchema='true').load('/FileStore/tables/dataSetEvents.csv')


In [2]:
from pyspark.sql.functions import split

split_col = split(event_df['timestamp'], ' ')
df = event_df.withColumn('Date', split_col.getItem(0))
df = event_df.withColumn('Time', split_col.getItem(1))

display(df)


In [3]:
import copy 

def event_split(x):
  y=copy.deepcopy(x).asDict()
  event=y['event'].replace("_"," ").split()
  y['event_action'] = event[0]
  target_string = event[1:len(event)]
  y['event_target'] = (" ".join(str(x) for x in target_string))
  return y

In [4]:
event_details = df.select("event").rdd.map(event_split).toDF().distinct()
display(event_details)

In [5]:
df_2 = df.join(event_details, on='event', how='inner').distinct()
display(df_2)

In [6]:
#From Prof. Franke 
from pyspark.sql import Row
def mapRow1(aRow):
  newRow = aRow.asDict()
  newRow['event'] = aRow['event'].split()[0]
  return Row(**newRow)

def mapRow29(aRow):
  newRow = Row(vin = aRow['vin'], event = aRow['event']).split()[0], count = aRow['count']
  return newRow

In [7]:
mapRow1(display(df))

In [8]:
split_col_2 = pyspark.sql.functions.split(df_2['timestamp'], ' ')
df_2 = df_2.withColumn('Date', split_col_2.getItem(0))
df_2 = df_2.withColumn('Time', split_col_2.getItem(1))
display(df_2)

In [9]:
from pyspark.sql.functions import concat
from pyspark.sql.functions import lit 

df_2 = df_2.withColumn('vin_prefix', concat(df_2.vin.substr(1,8), lit('_'), df_2.vin.substr(10,1)) )

In [10]:
display(df_2)

In [11]:
YMM = df_2.select('vin_prefix', 'year', 'make', 'model').distinct().orderBy('make','model','year','vin_prefix')
display(YMM)


In [12]:
YMM.repartition(1).write.format("com.databricks.spark.csv").options(header="true").save("/mnt/%s/Output/Spark-HW4/try8" %MOUNT_NAME);

In [13]:
VIN = df_2.select('vin', 'vin_prefix', 'trim', 'body_style', 'cab_style').distinct().orderBy('vin')
display(VIN)


In [14]:
VIN.repartition(1).write.format("com.databricks.spark.csv").options(header="true").save("/mnt/%s/Output/Spark-HW4/try9" %MOUNT_NAME);

In [15]:
location = df.select('zip', 'city', 'state').distinct().orderBy('zip', 'city', 'state')


In [16]:
display(location)

In [17]:
location.repartition(1).write.format("com.databricks.spark.csv").options(header="true").save("/mnt/%s/Output/Spark-HW4/try10" %MOUNT_NAME);

In [18]:
#Assumption: location = zip 
listing = df_2.select('vin', 'Date', 'zip','condition', 'price', 'mileage', 'image_count').distinct().orderBy('vin', 'Date')
display(listing)


In [19]:
listing.repartition(1).write.format("com.databricks.spark.csv").options(header="true").save("/mnt/%s/Output/Spark-HW4/try11" %MOUNT_NAME);

In [20]:
event = df_2.select('user_id', 'vin', 'Date', 'referring_domain', 'event_action', 'event_target').groupBy('user_id', 'vin', 'Date', 'referring_domain', 'event_action', 'event_target').count().orderBy('user_id', 'vin', 'Date', 'referring_domain', 'event_action', 'event_target')
display(event)

In [21]:
event.repartition(1).write.format("com.databricks.spark.csv").options(header="true").save("/mnt/%s/Output/Spark-HW4/try13" %MOUNT_NAME);

In [22]:
advertising_df = df_2.join(event, on=[df_2.user_id==event.user_id], how='inner').drop(event.user_id).drop(event.vin).drop(event.referring_domain)
display(advertising_df)

In [23]:
advertising_count = advertising_df.groupBy('user_id', 'vin').count().orderBy('user_id')
display(advertising_count)

In [24]:
print advertising_count.count()

In [25]:
advDF2= df_2.select('user_id', 'vin', 'condition', 'price', 'mileage', 'year').distinct()
advDF2=advDF2.join(advertising_count, on=[advDF2.user_id==advertising_count.user_id, advDF2.vin==advertising_count.vin], how='inner').drop(advertising_count.user_id).drop(advertising_count.vin)
display(advDF2)

In [26]:
print advDF2.count()

In [27]:
advDF2.repartition(1).write.format("com.databricks.spark.csv").options(header="true").save("/mnt/%s/Output/Spark-HW4/try12" %MOUNT_NAME);

In [28]:
joined_listing = advertising_count.join(listing, on=[advertising_count.vin == listing.vin], how='inner').drop(listing.image_count).drop(listing.vin)
joined_listing.count()
joined_vin = joined_listing.join(VIN, on = [joined_listing.vin==VIN.vin], how='inner').drop(joined_listing.vin).drop(VIN.body_style).drop(VIN.cab_style).drop(VIN.trim)
display(joined_vin)
joined_ymm = joined_vin.join(YMM, on=[YMM.vin_prefix == joined_vin.vin_prefix], how='inner')

print joined_ymm.count()